In [168]:
import pandas as pd
import datetime as dt
import seaborn as sns
import numpy as np

In [169]:
#行動データ
work_detail = pd.read_csv('../../datas/groupby_widedata/work_detail.csv')
work_env = pd.read_csv('../../datas/groupby_widedata/work_env.csv')
#fitbitデータ
calories = pd.read_csv('../../datas/fitbit_preprocessed_data/agg_for_day/calories_day.csv')
floors = pd.read_csv('../../datas/fitbit_preprocessed_data/agg_for_day/floors_day.csv')
heart = pd.read_csv('../../datas/fitbit_preprocessed_data/agg_for_day/heart_day.csv')
minutesFairlyActive= pd.read_csv('../../datas/fitbit_preprocessed_data/agg_for_day/minutesFairlyActive_day.csv')
minutesLightlyActive = pd.read_csv('../../datas/fitbit_preprocessed_data/agg_for_day/minutesLightlyActive_day.csv')
minutesSedentary = pd.read_csv('../../datas/fitbit_preprocessed_data/agg_for_day/minutesSedentary_day.csv')
minutesVeryActive = pd.read_csv('../../datas/fitbit_preprocessed_data/agg_for_day/minutesVeryActive_day.csv')
steps = pd.read_csv('../../datas/fitbit_preprocessed_data/agg_for_day/steps_day.csv')
sleep = pd.read_csv('../../datas/fitbit_preprocessed_data/sleep.csv')
#アンケートデータ
morning_bodyfatigue = pd.read_csv('./analyse_data/questionaire_data/morning_bodyfatigue.csv')
morning_dams = pd.read_csv('./analyse_data/questionaire_data/morning_dams.csv')
night_w_bodyfatigue = pd.read_csv('./analyse_data/questionaire_data/night_w_bodyfatigue.csv')
night_w_dams = pd.read_csv('./analyse_data/questionaire_data/night_w_dams.csv')
night_w_eval = pd.read_csv('./analyse_data/questionaire_data/night_w_eval.csv')
night_w_productivity = pd.read_csv('./analyse_data/questionaire_data/night_w_productivity.csv')
night_w_recover = pd.read_csv('./analyse_data/questionaire_data/night_w_recover.csv')
night_w_workenga = pd.read_csv('./analyse_data/questionaire_data/night_w_workenga.csv')
#天気データ
weather = pd.read_csv('../../datas/attribute_weater.csv')

In [170]:
night_w_dams['positive_score'] = night_w_dams['haturatu']+night_w_dams['tanosii']+night_w_dams['uresii']/3
night_w_dams['depressive_score'] = night_w_dams['kurai']+night_w_dams['iyana']+night_w_dams['sizunda']/3
night_w_dams['anxious_score'] = night_w_dams['kigakari']+night_w_dams['fuan']+night_w_dams['sinpai']/3

In [171]:
def to_datetime_jp2(x, datetime):
    x.index = pd.DatetimeIndex(x[datetime], name=datetime).tz_convert('Asia/Tokyo')
    x[datetime] = x.index
    x = x.reset_index(drop=True)
#     x[datetime] = x[datetime]+timedelta(days=1)
    x['date'] = x[datetime].dt.date
    return x

In [172]:
activity = activity.reset_index(drop=True)

In [173]:
value_unique = activity['activity'].unique()
zero_td = dt.timedelta(days=0, hours=0, minutes=0,
                        seconds=0, milliseconds=0, microseconds=0)
activity['started_at'] = pd.to_datetime(activity['started_at'])
activity['finished_at'] = pd.to_datetime(activity['finished_at'])
for i in range(len(value_unique)):
    activity['{}'.format(value_unique[i])] = zero_td  #timedeltaの０で埋める
    activity.loc[activity['activity']==value_unique[i],value_unique[i]]=activity['finished_at']-activity['started_at']  #行動の継続時間を計算
    activity['{}'.format(value_unique[i])] = activity['{}'.format(value_unique[i])].dt.total_seconds()/60  #
activity['date'] = activity['finished_at'].dt.date

KeyError: 'activity'

In [180]:
activity = activity.groupby(['date','user_id'],as_index=False).sum()

## アンケートデータの前処理

In [174]:
#目的変数のラベルづけ
#下位40%と上位40％の二値分類をする
def labeling(df, colmn):
    under_40 = df[colmn].quantile(0.4)
    upper_60 = df[colmn].quantile(0.6)
    df.loc[df[colmn]<=under_40,colmn + '_label']=0
    df.loc[df[colmn]>=upper_60,colmn + '_label']=2
    df.loc[(df[colmn]<upper_60)&(df[colmn]>under_40),colmn + '_label']=1

In [237]:
# def hour9():
attribute_weather = pd.read_csv('../../datas/attribute_weater.csv')
attribute_weather = attribute_weather[['Date', 'ave_temperature', 'max_temperature',
       'mini_temperature', 'sum_rainfall', 'sunlight_time', 'ave_windspeed',
       'max_windspeed', 'ave_vaporpressure', 'ave_cloudcover', 'weather_6to18',
       'weather_18to6', 'locate', 'user_id', 'sex', 'age']]

In [176]:
labeling(morning_bodyfatigue, 'value_hukai')
labeling(morning_bodyfatigue, 'value_boyake')
labeling(morning_bodyfatigue, 'value_nemuke')
labeling(morning_dams, 'positive_score')
labeling(morning_dams, 'depressive_score')
labeling(morning_dams, 'anxious_score')

labeling(night_w_bodyfatigue, 'value_hukai')
labeling(night_w_bodyfatigue, 'value_boyake')
labeling(night_w_bodyfatigue, 'value_nemuke')
labeling(night_w_dams, 'positive_score')
labeling(night_w_dams, 'depressive_score')
labeling(night_w_dams, 'anxious_score')
#夜アンケー自己評価
labeling(night_w_eval, 'kouritu')
labeling(night_w_eval, 'syutyu')
labeling(night_w_eval, 'yoteidori')
labeling(night_w_eval, 'jubun_comu')
labeling(night_w_eval, 'kouritu_comu')
labeling(night_w_eval, 'idea')
labeling(night_w_eval, 'seika')
#夜アンケー生産性
labeling(night_w_productivity, 'study_new')
labeling(night_w_productivity, 'decide_doing')
labeling(night_w_productivity, 'paformance')
labeling(night_w_productivity, 'absent_by_problem')
#夜アンケーリカバリー経験
labeling(night_w_recover, 'value_recover')
#夜アンケーワークエンゲージメント
labeling(night_w_workenga, 'workenga_score')

## それぞれのデータをマージ

In [177]:
#いらないカラムを捨てる
work_detail = work_detail[['date', 'user_id', 
       '管理者兼作業者', '企画業務', 'ノンコア業務（自身が本来担当ではない業務）', '管理者',
       '比較的慣れており，定型化できている業務', 'コア業務（自身が本来担当している業務）', '共同作業者', '開発業務', '管理業務',
       '作業者/参加者', '管理の業務', '比較的新規性が高く，試行錯誤を伴う定型化できていない業務', '営業業務',
       '事務業務（サポート業務、経理業務など）', '現場業務（主に肉体労働を伴うもの）', '現場（主に肉体労働を伴うもの）']]
work_env = work_env[['date', 'user_id',
       '家（その他）', 'どちらでもない', '一人', '家（書斎など執務向けの場所）', '他者あり（同僚など）', '家（リビングなど）',
       '快適', '職場外のワークスペース', '他者あり（無干渉）', '他者あり（同僚）', '職場', '他者あり（家族など）', '店/外',
       '不快', '非常に快適', '職場外ワークスペース', '非常に不快']]
calories = calories[[ 'user_id', 'date', 'mets_mean', 'calory_mean',
       'mets_median', 'calory_median', 'mets_var', 'calory_var', 'date_plus1']]
floors = floors[[ 'user_id', 'date', 'floor_mean', 'floor_median',
       'floor_var', 'date_plus1']]
heart = heart[['user_id', 'date', 'heart_mean', 'heart_median',
       'heart_var', 'date_plus1']]
minutesFairlyActive = minutesFairlyActive[['user_id', 'date', 'minutesFairlyActive_mean',
       'minutesFairlyActive_median', 'minutesFairlyActive_var', 'date_plus1']]
minutesLightlyActive = minutesLightlyActive[['user_id', 'date', 'minutesLightlyActive_mean',
       'minutesLightlyActive_median', 'minutesLightlyActive_var',
       'date_plus1']]
minutesSedentary = minutesSedentary[['user_id', 'date', 'minutesSedentary_mean',
       'minutesSedentary_median', 'minutesSedentary_var', 'date_plus1']]
minutesVeryActive = minutesVeryActive[[ 'user_id', 'date', 'minutesVeryActive_mean',
       'minutesVeryActive_median', 'minutesVeryActive_var', 'date_plus1']]
steps = steps[[ 'user_id', 'date', 'step_mean', 'step_median', 'step_var',
       'date_plus1']]
sleep = sleep[['Unnamed: 0', 'date', 'user_id', 'sleep', 'Unnamed: 0.1', 'datetime',
       'dateTime', 'sleep_1', 'sleep_2', 'sleep_3']]
morning_bodyfatigue = morning_bodyfatigue[[ 'date', 'user_id', 'value_nemuke',
       'value_hukai', 'value_boyake', 'value_nemuke_label',
       'value_hukai_label', 'value_boyake_label']]
morning_dams = morning_dams[[ 'user_id', 'Date_jp',  'date',
       'positive_score_label', 'depressive_score_label',
       'anxious_score_label']]
night_w_bodyfatigue = night_w_bodyfatigue[['user_id', 'date',
       'value_hukai_label', 'value_boyake_label', 'value_nemuke_label']]
night_w_dams = night_w_dams[[ 'date', 'user_id', 'positive_score_label',
       'depressive_score_label', 'anxious_score_label']]
night_w_eval = night_w_eval[['datetime_jp', 'user_id',
       'value_eval', 'hour', 
       'date', 'kouritu_label', 'syutyu_label', 'yoteidori_label',
       'jubun_comu_label', 'kouritu_comu_label', 'idea_label', 'seika_label']]
night_w_productivity = night_w_productivity[['datetime_jp', 'user_id',
       'date', 'study_new_label', 'decide_doing_label', 'paformance_label',
       'absent_by_problem_label']]
night_w_recover = night_w_recover[['datetime_jp', 'user_id', 
       'date', 'value_recover_label']]
night_w_workenga = night_w_workenga[['datetime_jp', 'user_id', 
        'date','workenga_score_label']]


In [178]:
activity = activity[[ 'user_id','date', '食事（業務時間外）',
       '単独作業', '趣味・休憩', '家事・育児', '休憩（業務時間内）', '移動', 'Web会議', 'その他', '業務外作業',
       '対面会議', '共同作業（コミュニケーションを伴う）', '電話（会議）']]

In [208]:
# アクティビティデータのマージ
merged_act = pd.merge(activity_1, work_detail_1, on=['date','user_id'],how='left')
merged_act = pd.merge(merged_act, merged_act, on=['date', 'user_id'],how='left')

# work_env.to_csv('./work_data/work_env.csv')

In [222]:
#fitbitデータ
merged_fitbit = pd.merge(calories, floors, on=['date', 'user_id'], how='left')
merged_fitbit = pd.merge(merged_fitbit, heart, on=['date', 'user_id'], how='left')
merged_fitbit = pd.merge(merged_fitbit, minutesFairlyActive, on=['date', 'user_id'], how='left')
merged_fitbit = pd.merge(merged_fitbit, minutesLightlyActive, on=['date', 'user_id'], how='left')
merged_fitbit = pd.merge(merged_fitbit, minutesSedentary, on=['date', 'user_id'], how='left')
merged_fitbit = pd.merge(merged_fitbit, minutesVeryActive, on=['date', 'user_id'], how='left')
merged_fitbit = pd.merge(merged_fitbit, steps, on=['date', 'user_id'], how='left')
merged_fitbit = pd.merge(merged_fitbit, sleep, on=['date', 'user_id'], how='left')
# calories.to_csv('./fitbit/calories.csv')
# floors.to_csv('./fitbit/floors.csv')
# heart.to_csv('./fitbit/heart.csv')
# minutesFairlyActive.to_csv('./fitbit/minutesFairlyActive.csv')
# minutesLightlyActive.to_csv('./fitbit/minutesLightlyActive.csv')
# minutesSedentary.to_csv('./fitbit/minutesSedentary.csv')
# minutesVeryActive.to_csv('./fitbit/minutesVeryActive.csv')
# steps.to_csv('./fitbit/steps.csv')
# sleep.to_csv('./fitbit/sleep.csv')

In [232]:
#アンケートデータのマージ
#朝
merged_morning = pd.merge(morning_dams, morning_bodyfatigue, on=['date','user_id'], how='left')
#平日夜
merged_night_w = pd.merge(night_w_dams, night_w_bodyfatigue, on=['date','user_id'], how='left')
merged_night_w = pd.merge(merged_night_w, night_w_eval, on=['date','user_id'], how='left')
merged_night_w = pd.merge(merged_night_w, night_w_productivity, on=['date','user_id'], how='left')
merged_night_w = pd.merge(merged_night_w, night_w_recover, on=['date','user_id'], how='left')
merged_night_w = pd.merge(merged_night_w, night_w_workenga, on=['date','user_id'], how='left')

In [235]:
# merged_night_w.to_csv('./merged_data/merged_night_w.csv')
# merged_fitbit.to_csv('./merged_data/merged_fitbit.csv')
# merged_act.to_csv('./merged_data/merged_act.csv')


In [ ]:
# def lgbm_model()
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# ques_act_day_pred['user_id'] = ques_act_day_pred['user_id'].astype('int')
X = 
y = 

# 学習データとテストデータを分ける
X_train, X_test, y_train, y_test = train_test_split(X, y)

# データを格納する
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# 学習条件を設定
params = {'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'multiclass',
          'metric': {'multi_logloss'},
          'num_class': 3,
          'learning_rate': 0.02,
          'num_leaves': 23,
          'min_data_in_leaf': 1,
          'num_iteration': 1000,
          'verbose': 0}

# 学習する
model = lgb.train(params,
                  lgb_train,
                  num_boost_round=50,
                  valid_sets=lgb_eval,
                  early_stopping_rounds=20)

# テストデータで予測する
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1)

# Accuracy を計算する
accuracy = sum(y_test == y_pred_max) / len(y_test)
print('accuracy:', accuracy)

# importanceを表示する
importance = pd.DataFrame(model.feature_importance(), index=list(X.columns), columns=['importance'])
display(importance)
importance.plot.bar()
